**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Dec  1 02:36:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 1.LogoNUSEEP.png
'2ª APRESENTAÇÃO INSTITUCIONAL (1).pdf'
'2ª APRESENTAÇÃO INSTITUCIONAL.pdf'
'2. Missão visão e valores.png'
'3. Apresentação Institucional - NUSEEP.pdf'
'4.Regimento Interno NUSEEP 2021 .pdf'
'5.Planejamento Estratégico do NUSEEP 2021.pdf'
'6. Mapeamento de IES.png'
 A34A7597.JPG
 A34A7600.JPG
 A34A7707.JPG
 A34A7709.JPG
'A apresentação do NUSEEP 2021.2 (1).pdf'
'A apresentação do NUSEEP 2021.2.pdf'
'Análise Thaís'
'Antônio dos Santos Ramos Neto - Certidão de Quitação Eleitoral.pdf'
'Antônio dos Santos Ramos Neto - Comprovação Profissional.pdf'
'Antônio dos Santos Ramos Neto - Comprovante do GRU.pdf'
'Antônio dos Santos Ramos Neto - CPF.pdf'
'Antonio dos Santos Ramos Neto - Declaração de Concludente.pdf'
'Antônio dos Santos Ramos Neto - DISC.gsheet'
'Antônio dos Santos Ramos Neto - Histórico.pdf'
'Antônio dos Santos Ramos Neto - Iniciação Científica.pdf'
'Antônio dos Santos Ramos Neto - Produção Científic

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.16 MiB | 15.94 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
print('classes= 5\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3')

classes= 5
train  = data/train.txt
valid  = data/test.txt
names = data/obj.names
backup = /mydrive/yolov3


In [ ]:
!echo ["caneta","1","2","3","4"] > data/obj.names
!echo -e 'classes= 5\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-12-01 02:38:11--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  16.6MB/s    in 12s     

2022-12-01 02:38:23 (13.3 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/IMG_20221130_205627.jpg  
  inflating: data/obj/IMG_20221130_205627.txt  
  inflating: data/obj/IMG_20221130_205628.jpg  
  inflating: data/obj/IMG_20221130_205628.txt  
  inflating: data/obj/IMG_20221130_205629.jpg  
  inflating: data/obj/IMG_20221130_205629.txt  
  inflating: data/obj/IMG_20221130_205630.jpg  
  inflating: data/obj/IMG_20221130_205630.txt  
  inflating: data/obj/IMG_20221130_205634.jpg  
  inflating: data/obj/IMG_20221130_205634.txt  
  inflating: data/obj/IMG_20221130_205636.jpg  
  inflating: data/obj/IMG_20221130_205636.txt  
  inflating: data/obj/IMG_20221130_205755.jpg  
  inflating: data/obj/IMG_20221130_205755.txt  
  inflating: data/obj/IMG_20221130_205804.jpg  
  inflating: data/obj/IMG_20221130_205804.txt  
  inflating: data/obj/IMG_20221130_205808.jpg  
  inflating: data/obj/IMG_20221130_205808.txt  
  inflating: data/obj/IMG_20221130_205811.jpg  
  inflating: data/obj/IMG_20221130_205811.txt  
  i

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.505469 0.461719 0.842187 0.329688

['0', '0.505469', '0.461719', '0.842187', '0.329688']
0 data/obj/IMG_20221130_205804.txt
0 0.505469 0.461719 0.842187 0.329688
1 0.037500 0.035937 0.046875 0.046875

['1', '0.037500', '0.035937', '0.046875', '0.046875']
0 data/obj/IMG_20221130_205804.txt
0 0.037500 0.035937 0.046875 0.046875
2 0.962500 0.041406 0.040625 0.045312

['2', '0.962500', '0.041406', '0.040625', '0.045312']
0 data/obj/IMG_20221130_205804.txt
0 0.962500 0.041406 0.040625 0.045312
3 0.056250 0.959375 0.062500 0.050000

['3', '0.056250', '0.959375', '0.062500', '0.050000']
0 data/obj/IMG_20221130_205804.txt
0 0.056250 0.959375 0.062500 0.050000
4 0.949219 0.954688 0.057813 0.046875

['4', '0.949219', '0.954688', '0.057813', '0.046875']
0 data/obj/IMG_20221130_205804.txt
0 0.949219 0.954688 0.057813 0.046875
0 0.525000 0.411719 0.359375 0.560937

['0', '0.525000', '0.411719', '0.359375', '0.560937']
1 data/obj/IMG_20221130_210407.txt
0 0.525000 0.411719 0.359375 0.560937
1 0.

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/IMG_20221130_210218.jpg', 'data/obj/IMG_20221130_210241.jpg', 'data/obj/IMG_20221130_210204.jpg', 'data/obj/IMG_20221130_205849.jpg', 'data/obj/IMG_20221130_210200.jpg', 'data/obj/IMG_20221130_210243.jpg', 'data/obj/IMG_20221130_210359.jpg', 'data/obj/IMG_20221130_210203.jpg', 'data/obj/IMG_20221130_205852.jpg', 'data/obj/IMG_20221130_210259.jpg', 'data/obj/IMG_20221130_210119.jpg', 'data/obj/IMG_20221130_210041.jpg', 'data/obj/IMG_20221130_210206.jpg', 'data/obj/IMG_20221130_205838.jpg', 'data/obj/IMG_20221130_205843.jpg', 'data/obj/IMG_20221130_210018.jpg', 'data/obj/IMG_20221130_210230.jpg', 'data/obj/IMG_20221130_205804.jpg', 'data/obj/IMG_20221130_210009.jpg', 'data/obj/IMG_20221130_205755.jpg', 'data/obj/IMG_20221130_210217.jpg', 'data/obj/IMG_20221130_210351.jpg', 'data/obj/IMG_20221130_205811.jpg', 'data/obj/IMG_20221130_210255.jpg', 'data/obj/IMG_20221130_205950.jpg', 'data/obj/IMG_20221130_205902.jpg', 'data/obj/IMG_20221130_210207.jpg', 'data/obj/IMG_20221130_2103

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!ls data/obj

IMG_20221130_205627.jpg  IMG_20221130_210013.jpg  IMG_20221130_210239.jpg
IMG_20221130_205627.txt  IMG_20221130_210013.txt  IMG_20221130_210239.txt
IMG_20221130_205628.jpg  IMG_20221130_210018.jpg  IMG_20221130_210241.jpg
IMG_20221130_205628.txt  IMG_20221130_210018.txt  IMG_20221130_210241.txt
IMG_20221130_205629.jpg  IMG_20221130_210033.jpg  IMG_20221130_210243.jpg
IMG_20221130_205629.txt  IMG_20221130_210033.txt  IMG_20221130_210243.txt
IMG_20221130_205630.jpg  IMG_20221130_210037.jpg  IMG_20221130_210245.jpg
IMG_20221130_205630.txt  IMG_20221130_210037.txt  IMG_20221130_210245.txt
IMG_20221130_205634.jpg  IMG_20221130_210039.jpg  IMG_20221130_210247.jpg
IMG_20221130_205634.txt  IMG_20221130_210039.txt  IMG_20221130_210247.txt
IMG_20221130_205636.jpg  IMG_20221130_210041.jpg  IMG_20221130_210250.jpg
IMG_20221130_205636.txt  IMG_20221130_210041.txt  IMG_20221130_210250.txt
IMG_20221130_205755.jpg  IMG_20221130_210059.jpg  IMG_20221130_210255.jpg
IMG_20221130_205755.txt  IMG_20221130_

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 636231, rewritten_bbox = 0.000000 % 
]2;3926/4000: loss=0.4 hours left=0.3
 3926: 0.381845, 0.384587 avg loss, 0.001000 rate, 9.736248 seconds, 251264 images, 0.259585 hours left
Loaded: 0.000118 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.902022), count: 4, class_loss = 0.068435, iou_loss = 0.032543, total_loss = 0.100978 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000295, iou_loss = 0.000000, total_loss = 0.000295 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.808423), count: 8, class_loss = 0.433804, iou_loss = 0.290281, total_loss = 0.724085 
 total_bbox = 636243, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.875559), count: 4, class_loss = 0.000006, iou_loss = 0.050979, total_loss = 0.050985 
v3 (m